In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy.spatial.distance import cdist
import tqdm
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
import shapely
from numpy.random import default_rng

In [2]:
vert_df = pd.read_csv("../data/processed/ed_soa_data_frame.csv")
vert_df = gpd.GeoDataFrame(vert_df)

vert_df['geometry'] = vert_df['geometry'].apply(shapely.wkt.loads)
vert_df['centroid'] = vert_df['centroid'].apply(shapely.wkt.loads)

ed_travel_df = pd.read_csv("../data/processed/ed_travel_data.csv")
pop_ratio_mat = np.loadtxt('../data/processed/ed_soa_pop_ratio_mat.csv', delimiter=',')
dist_mat = np.loadtxt('../data/processed/ed_soa_dist_mat.csv', delimiter=',')
prob_mat = np.loadtxt('../data/processed/ed_soa_travel_prob_mat.csv', delimiter=',')

In [3]:
pop = vert_df.population
commuting_pop = pop*0.6
#no_commuters = ed_travel_df['No. of Commuters'].to_numpy()
#pop_ratio = ed_travel_df['pop_ratio'].to_numpy()
#dist = ed_travel_df['Distance'].to_numpy()

#dist = np.repeat(dist, no_commuters)
#log_dist = np.log10(dist)
#pop_ratio = np.repeat(pop_ratio, no_commuters)
#log_pop_ratio = np.log10(pop_ratio)

In [4]:
state_mat = np.zeros((vert_df.population.size, 5))

In [5]:
state_mat[1000, 0] -= 2
state_mat[1000, 2] += 2

In [6]:
rng = default_rng()

In [7]:
travel_mat = np.zeros_like(prob_mat)
for row in range(prob_mat.shape[0]):
    idx = rng.choice(range(prob_mat.shape[1]), int(pop[row]*0.6), replace = True, p = prob_mat[row,:])
    travel_mat[row] += np.bincount(idx, minlength = prob_mat.shape[1])

In [75]:
def choice_bincount(rng, size=None, replace=True, p=None, axis=0, shuffle=True):
    idx = rng.choice(range(p.size), size, replace, p, axis, shuffle)
    return np.bincount(idx, minlength = p.size)

In [164]:
%%time
travel_mat = np.zeros_like(prob_mat)
for row in range(prob_mat.shape[0]):
    travel_mat[row] = choice_bincount(rng, int(pop[row]*0.6), p = prob_mat[row])

CPU times: user 1.81 s, sys: 0 ns, total: 1.81 s
Wall time: 1.81 s


In [93]:
from numpy.random import SeedSequence, default_rng

ss = SeedSequence(12345)
child_seeds = ss.spawn(prob_mat.shape[0])
streams = [default_rng(s) for s in child_seeds]

In [82]:
travel_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
from multiprocessing import Pool

In [189]:
travel_mat = np.zeros_like(prob_mat)

In [202]:
%%time

pool = Pool(processes=16)

res = []

def update_tavel_mat(x):
    res.append(x)
    
def choice_bincount_order(x, y):
    return x, choice_bincount(*y)

[pool.apply_async(choice_bincount_order, 
                args = (row, (default_rng(streams[row]), int(pop[row]*0.6), True, prob_mat[row])),
                callback = update_tavel_mat) for row in range(prob_mat.shape[0])]
    
pool.close()
pool.join()  

#for i in res:
#    travel_mat[i[0]] = i[1]

CPU times: user 577 ms, sys: 423 ms, total: 1 s
Wall time: 873 ms


In [203]:
travel_mat.sum()

3941934.0

In [10]:
def nd_rand_integers(x, vals, p):
    
    assert(x.size == p.shape[1])
    
    rng = default_rng()
    travel_mat = np.zeros(prob_mat.shape(), dtype = vals.dtype)
    
    if len(vals.shape) == 1:
    
        for row in range(p.shape[0]):
            idx = rng.choice(range(p.shape[1]), x, replace = True, p = p[row,:])
            travel_mat[row] += np.bincount(idx, minlength = prob_mat.shape[1])


IndentationError: expected an indented block (<ipython-input-10-3c9e46ad445a>, line 4)

In [12]:
np.array([0,0]).shape[1]

IndexError: tuple index out of range